## タスク 8.1: エージェントのノートブックを実行する

このノートブックでは、ショッピングアシスタントエージェントを使用して、AnyCompany Outdoor Power Equipment と AnyCompany LawnCare Solutions の 2 社の芝生メンテナンス製品に関する質問に答えます。このノートブックのセルを実行して、エージェントとのチャット会話をシミュレートし、事前に作成済みのナレッジベース、ガードレール、エージェントアプリケーションをテストします。

ナレッジベースには、メーカー、説明、評価などの製品の詳細が含まれています。エージェントは、簡単な計算と価格の検索をサポートする機能にアクセスできます。

次のステップでは、エージェントを一覧表示し、セッションとメモリのパラメータを使用してショッピングアシスタントエージェントを呼び出して、エージェントの行動に対する短期記憶と長期記憶の影響について理解します。また、トレースをキャプチャして調査し、ガードレール付きのショッピングアシスタントの使用など、エージェントの機能について理解します。

ノートブックの各セルに進み、各コードセルを実行してその出力を確認します。

### タスク 8.1.1: Boto3 のインストールと必要な Python ライブラリのインポート

このタスクでは、Boto3 とサポート用の Python ライブラリをインポートします。また、Boto3 クライアントエラー処理ライブラリをインポートし、セッション、AWS リージョン、Bedrock クライアント、ヘルパーエージェント関数を設定します。

次のセルは Python Boto3 ライブラリをインポートします。
AWS SDK for Python (Boto3) は、AWS インフラストラクチャサービス用の Python API を提供します。SDK for Python を使用すると、AWS サービスの上にアプリケーションを構築できます。Boto3 ライブラリは Amazon Bedrock エージェントおよび基盤モデルと対話します。

In [ ]:
import boto3
import json
import time
import logging
import uuid
from botocore.exceptions import ClientError

### タスク 8.1.2: Boto3 オブジェクトとヘルパー関数を設定する 

セッション、AWS リージョン、Bedrock クライアントを設定します。

In [ ]:
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

`shopping-assistant` という名前のエージェントに関連付けられているエージェント ID を取得します。

In [ ]:
agentName = "shopping-assistant"

try:
    response = bedrock_agent_client.list_agents()
    agents = response.get('agentSummaries', [])
    agent_id = None
    for agent in agents:
        if agent['agentName'] == agentName:
            agent_id = agent['agentId']
    
    if agent_id == None:
        print(f"We couldn't find the Agent ID of the agent named {agentName}. Make sure that agent exists or modify the variable above with the right name for the agent you created.")
    else:
        print(f"Found the Agent {agentName} with Agent ID {agent_id}")
except ClientError as e:
    print(f"Error listing agents: {e}")

エイリアス名が `prod` のエージェントのエイリアス ID を検索します。

In [ ]:
agentAliasName = "prod"

try:
    # Try to get the alias ID for the shopping-assistant agent
    response = bedrock_agent_client.list_agent_aliases(agentId=agent_id)
    alias_id = None
    if response.get('agentAliasSummaries'):
        for alias in response['agentAliasSummaries']:
            if alias['agentAliasName'] == 'prod':
                alias_id = alias['agentAliasId']
                break
    if alias_id == None:
        print(f"We couldn't find the Alias ID of the agent named {agentName} with an alias name {agentAliasName}. Make sure that you deployed the agent under that alias name or modify the variable 'agentAliasName' above with the right name for the alias name you created.")
    print(f"Using agent alias ID: {alias_id}")
except Exception as e:
    print(f"Error getting agent alias: {e}")

Bedrock エージェントを呼び出して記憶をキャプチャするためのヘルパー関数を作成します。

In [ ]:
def invokeAgent(query, session_id, memory_id, agent_id, enable_trace=False, session_state=dict(), end_session=False):
    
    agentResponse = bedrock_agent_runtime_client.invoke_agent(
        inputText=query,
        agentId=agent_id,
        agentAliasId=alias_id, 
        memoryId=memory_id,
        sessionId=session_id,
        enableTrace=enable_trace, 
        endSession= end_session,
        sessionState=session_state
    )
    
    if enable_trace:
        # Create a safe copy of the response for logging, handling datetime objects
        try:
            # Log only the metadata, not the event stream
            safe_response = {k: str(v) for k, v in agentResponse.items() if k != 'completion'}
            logger.info(f"Agent response metadata: {safe_response}")
        except Exception as e:
            logger.info(f"Could not print full response due to: {str(e)}")
    
    event_stream = agentResponse['completion']
    agent_answer = ""
    try:
        for event in event_stream:
            if 'chunk' in event:
                data = event['chunk']['bytes']
                chunk_text = data.decode('utf8')
                agent_answer += chunk_text
            elif 'trace' in event:
                if enable_trace:
                    # Handle potential datetime objects in trace
                    try:
                        trace_str = json.dumps(event['trace'], default=lambda o: str(o), indent=2)
                        logger.info(trace_str)
                    except Exception as e:
                        logger.info(f"Could not print trace due to: {str(e)}")
                        logger.info(str(event['trace']))
            else:
                raise Exception("unexpected event.", event)
        
        if enable_trace:
            logger.info(f"Final answer ->\n{agent_answer}")
        return agent_answer
    except Exception as e:
        if str(e).find("throttlingException"): 
            print("\033[1mThrottlingException:\033[;7m\nA throttling error occurred when calling the InvokeAgent operation. Please wait up to 60 seconds and retry this cell.\n\033[0m")

### タスク 8.1.3: ガードレールを使わずに Bedrock エージェントと対話する  

Bedrock エージェントとやりとりしたりチャットしたりする場合、状態を維持するのに役立つ 2 つの要素があります。

**session_id** は、複数の質問にまたがるエージェントとの会話を表します。エージェントと同じ会話を続けるには、リクエストで同じ session_id 値を使用します。 

**memory_id** には、各ユーザーの会話履歴とコンテキストが安全に保存され、ユーザーどうしが完全に分離されます。 

**session_id** と **memory_id** を設定します。

In [ ]:
session_id_1:str = str(uuid.uuid4())
print("Session id="+session_id_1)
memory_id_1:str = str(uuid.uuid4())
print("Memory id="+memory_id_1)

エージェントとの対話をシミュレートします。エージェントに最初の質問をして、会話を始めます。エージェントへの質問の一部は、ナレッジベースに保存されている情報から回答されます。ロギングがアクティブなため、レイテンシーに気付くはずです。**shopping-assistant** エージェントが呼び出されていることに注意してください。このエージェントには関連付けられたガードレールがありません。

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **注:** このラボでは、モデル呼び出しレート制限のある非本番環境の AWS アカウントを使用します。この後、セルでエージェントに質問を投稿して**スロットリングエラー**になる場合は、最大 60 秒待ってからそのセルを再実行してください。

In [ ]:
q="こんにちは。貴社の商品に興味があります。貴社が販売している商品の製造元を教えてください。"
print("\033[1mQuestion:\033[0m "+q+"\n")
try: 
    response = invokeAgent(q, session_id_1, memory_id_1, agent_id)
    print("\033[1mResponse:\033[0m "+response+"\n")
except Exception as e:
    print(f"Error: {str(e)}")

製品に関する質問を続けます。各質問を実行し、応答を確認します。

In [ ]:
q="「String Trimmer」の価格を教えてください。"
print("\033[1mQuestion:\033[0m "+q+"\n")
try: 
    response = invokeAgent(q, session_id_1, memory_id_1, agent_id)
    print("\033[1mResponse:\033[0m "+response+"\n")
except Exception as e:
    print(f"Error: {str(e)}")

別の質問をします。これは、数学演算を使用するようエージェントに指示するものです。

In [ ]:
q="「String Trimmer」を2つ購入するといくらかかりますか?"
print("\033[1mQuestion:\033[0m "+q+"\n")
try: 
    response = invokeAgent(q, session_id_1, memory_id_1, agent_id)
    print("\033[1mResponse:\033[0m "+response+"\n")
except Exception as e:
    print(f"Error: {str(e)}")

このセッションの最後に、ガードレールによってブロックされる質問をします。

In [ ]:
q="Can I use a string trimmer to control weeds?"
print("\033[1mQuestion:\033[0m "+q+"\n")
try: 
    response = invokeAgent(q, session_id_1, memory_id_1, agent_id)
    print("\033[1mResponse:\033[0m "+response+"\n")
except Exception as e:
    print(f"Error: {str(e)}")

以下のコードで **end_session=True** を設定してセッションを終了し、後でメモリストアからセッションサマリーにアクセスできるようにします。 

In [ ]:
try:
    response = invokeAgent("end session", session_id_1, memory_id_1, agent_id, end_session=True)
    print("\033[1mResponse:\033[0m "+response+"\n")
except Exception as e:
    print(f"Error: {str(e)}")

### タスク 8.1.4: トレースの使用

このセクションでは、同じプロンプトを送信しますが、今回はトレースを使用します。トレースの出力を分析することで、エージェントがどのように考えているのかを明らかにすることが目的です。

まず、新しい session_id で最初からやり直しましょう。ただし、次のセクションで再利用されるので、memory_id は同じままにしておきます。

In [ ]:
session_id_2:str = str(uuid.uuid4())
print("Session id="+session_id_2)
print("Keeping the same memory id="+memory_id_1)

この最初のプロンプトでは、ナレッジベースで情報を確認してください。

In [ ]:
q="こんにちは。貴社の商品に興味があります。貴社が販売している商品の製造元を教えてください。"
print("\033[1mQuestion:\033[0m "+q+"\n")
try: 
    response = invokeAgent(q, session_id_2, memory_id_1, agent_id, enable_trace=True)
    print("\033[1mResponse:\033[0m "+response+"\n")
except Exception as e:
    print(f"Error: {str(e)}")

この 2 番目のプロンプトは、アクショングループ (Lambda) を呼び出します。LLM がどのタイミングでアクショングループ/ツールの使用を決定したか、また、最終的な回答を作成するために LLM に送り返される前のツールからの回答は何だったかを確認できますか。

In [ ]:
q="「String Trimmer」の価格を教えてください。"
print("\033[1mQuestion:\033[0m "+q+"\n")
try: 
    response = invokeAgent(q, session_id_2, memory_id_1, agent_id, enable_trace=True)
    print("\033[1mResponse:\033[0m "+response+"\n")
except Exception as e:
    print(f"Error: {str(e)}")

### タスク 8.1.5: エージェントのセッションメモリと対話する

最初のセッションを終了してから 2 分以上が経過しました。これで、最初のセッションのサマリーが記述されたメモリストアの準備ができているはずです。これは、複数の会話にわたるエージェントの推論を監査、デバッグ、または理解するのに役立ちます。以下の関数は、セッションサマリーが確定されているかどうかを確認し、セッションサマリーを出力します。サマリーがまだ利用できない場合、以下のコードは 30 秒間待機し、最大 5 回再試行します。このセルを実行した後、サマリーが表示されない場合は、もう 1 分待ってからこのセルを再実行します。

In [ ]:
def check_session_summary(bedrock_agent_runtime_client, agent_id, alias_id, memory_id, max_retries=5, wait_time=30):
    for attempt in range(max_retries):
        try:
            response = bedrock_agent_runtime_client.get_agent_memory(
                agentId=agent_id,
                agentAliasId=alias_id,
                memoryId=memory_id,
                memoryType='SESSION_SUMMARY'
            )
            
            if 'memoryContents' in response and response['memoryContents']:
                print("Summary found!")
                return response['memoryContents'][0]['sessionSummary']['summaryText']
            
            print(f"Attempt {attempt + 1}/{max_retries}: Summary not yet generated. Waiting {wait_time} seconds...")
            time.sleep(wait_time)
            
        except Exception as e:
            print(f"Error checking summary: {str(e)}")
            return None
    
    print("Summary not found after all retries. The summary might take longer to generate.")
    print("Please wait 60 seconds and rerun this cell.")
    return None

# Usage example
try:
    summary = check_session_summary(
        bedrock_agent_runtime_client,
        agent_id=agent_id,
        alias_id=alias_id,
        memory_id=memory_id_1
    )

    if summary:
        print("\nSession Summary:")
        print(summary)
except Exception as e:
    print(f"Error retrieving session summary: {str(e)}")

話し合った内容を振り返るようにエージェントに依頼して、この会話を終了します。

In [ ]:
q='私は何の商品を買うことに興味があったのでしょうか?'
print("\033[1mQuestion:\033[0m "+q+"\n")
try: 
    response = invokeAgent(q, session_id_1, memory_id_1, agent_id)
    print("\033[1mResponse:\033[0m "+response+"\n")
except Exception as e:
    print(f"Error: {str(e)}")

### まとめ

このノートブックを完了しました。ラボの次の部分に進むために、以下を実行してください。
- このノートブックファイルを閉じる。
- ラボのセッションに戻り、「**まとめ**」に進む。